---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [1]:
import pandas as pd
import numpy as np

In [2]:
train = pd.read_csv('train.csv', engine='python')
test = pd.read_csv('test.csv', engine='python')
test_id = pd.read_csv('test.csv', engine='python')['ticket_id']
lat = pd.read_csv('latlons.csv', engine='python')
addr = pd.read_csv('addresses.csv', engine='python')   

In [3]:
train.head(2)

,ticket_id,agency_name,inspector_name,violator_name,violation_street_number,violation_street_name,violation_zip_code,mailing_address_str_number,mailing_address_str_name,city,...,clean_up_cost,judgment_amount,payment_amount,balance_due,payment_date,payment_status,collection_status,grafitti_status,compliance_detail,compliance
0,22056,"Buildings, Safety Engineering & Env Department","Sims, Martinzie","INVESTMENT INC., MIDWEST MORTGAGE",2900.0,TYLER,NaN,3.0,S. WICKER,CHICAGO,...,0.0,305.0,0.0,305.0,NaN,NO PAYMENT APPLIED,NaN,NaN,non-compliant by no payment,0.0
1,27586,"Buildings, Safety Engineering & Env Department","Williams, Darrin","Michigan, Covenant House",4311.0,CENTRAL,NaN,2959.0,Martin Luther King,Detroit,...,0.0,855.0,780.0,75.0,2005-06-02 00:00:00,PAID IN FULL,NaN,NaN,compliant by late payment within 1 month,1.0


In [4]:
df = pd.concat([train, test])

In [5]:
df = pd.merge(df,addr, on='ticket_id')

In [6]:
df = pd.merge(df,lat,on='address', how='left')

In [7]:
df.corr()

,ticket_id,violation_street_number,fine_amount,admin_fee,state_fee,late_fee,discount_amount,clean_up_cost,judgment_amount,payment_amount,balance_due,compliance,lat,lon
ticket_id,1.000000,0.006974,-0.046480,0.199787,0.199787,0.013960,-0.001850,0.062105,0.029276,-0.048120,0.003718,-0.026264,0.030173,-0.005515
violation_street_number,0.006974,1.000000,-0.001062,0.004531,0.004531,0.000759,-0.000026,-0.001149,0.000392,-0.000514,0.003301,-0.002526,0.053010,-0.031671
fine_amount,-0.046480,-0.001062,1.000000,-0.049492,-0.049492,0.726691,0.028407,-0.009336,0.714373,0.185201,0.677729,-0.049134,-0.011119,-0.010205
admin_fee,0.199787,0.004531,-0.049492,1.000000,1.000000,0.267349,0.027805,0.024061,0.303721,0.139338,0.275884,NaN,0.029466,-0.009080
state_fee,0.199787,0.004531,-0.049492,1.000000,1.000000,0.267349,0.027805,0.024061,0.303721,0.139338,0.275884,NaN,0.029466,-0.009080
late_fee,0.013960,0.000759,0.726691,0.267349,0.267349,1.000000,-0.018158,-0.002925,0.971661,0.198416,0.948404,-0.085055,0.004478,-0.014832
discount_amount,-0.001850,-0.000026,0.028407,0.027805,0.027805,-0.018158,1.000000,-0.000101,0.038637,0.148797,-0.018124,0.156073,-0.000801,-0.004222
clean_up_cost,0.062105,-0.001149,-0.009336,0.024061,0.024061,-0.002925,-0.000101,1.000000,0.176901,NaN,NaN,NaN,-0.008913,0.006222
judgment_amount,0.029276,0.000392,0.714373,0.303721,0.303721,0.971661,0.038637,0.176901,1.000000,0.244550,0.941940,-0.052637,0.000582,-0.014267
payment_amount,-0.048120,-0.000514,0.185201,0.139338,0.139338,0.198416,0.148797,NaN,0.244550,1.000000,-0.075703,0.213319,-0.011322,-0.014204


In [8]:
df.isnull().sum()[df.isnull().sum() != 0]

violator_name                     62
violation_zip_code            287283
mailing_address_str_number      4616
mailing_address_str_name           7
city                               1
state                            424
zip_code                           4
non_us_str_code               311304
hearing_date                   14688
fine_amount                        1
payment_amount                 61001
balance_due                    61001
payment_date                  270194
payment_status                 61001
collection_status             274410
grafitti_status               309085
compliance_detail              61001
compliance                    151427
lat                                8
lon                                8
dtype: int64

In [9]:
df = df.drop(['balance_due', 'city', 'collection_status', 'compliance_detail', 'grafitti_status', 'hearing_date',
        'mailing_address_str_number', 'non_us_str_code','payment_amount', 'payment_date','payment_status',
        'state', 'violation_zip_code', 'violator_name','address', 'ticket_id', 'inspector_name'], axis=1)
df = df.drop(['violation_description','mailing_address_str_name', 'judgment_amount','violation_street_name'],axis=1)
df = df.drop('zip_code',axis=1)

In [10]:
df['ticket_issued_date'] = pd.to_datetime(df['ticket_issued_date'])

df['month'] = df['ticket_issued_date'].apply(lambda x:x.month)
df['hour'] = df['ticket_issued_date'].apply(lambda x:x.hour)

df = df.drop('ticket_issued_date',axis=1)

In [11]:
obj_cols = list(df.select_dtypes(['object']).columns)

In [12]:
df = pd.get_dummies(df, columns=obj_cols, drop_first=True)

In [13]:
train_df = df[:-61001]
test_df = df[-61001:]

In [14]:

train_df['compliance'] = train_df['compliance'].fillna(-1)
train_df = train_df[train_df['compliance'] != -1]



C:\Users\Fida Hussain\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
test_df = test_df.drop('compliance', axis=1)

In [16]:
for i in [test_df, train_df]:
    for j in ['lat', 'lon']:
        i[j] = i[j].fillna(i[j].mean())



In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X = train_df.drop('compliance',axis=1)
y = train_df['compliance']

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=0)

In [26]:
from sklearn.ensemble import RandomForestClassifier
    
rf = RandomForestClassifier(250).fit(X_train, y_train)
score = rf.predict_proba(X_test)

In [27]:
from sklearn.metrics import roc_auc_score

In [28]:
print('AUC score : ', roc_auc_score(y_test, score[:,1]))

AUC score :  0.8020127534589898
